### create resources

In [1]:
import time

from azure.identity import AzureCliCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.resource import ResourceManagementClient

import tiktoken 

In [2]:
subscription_id  = '<your-subscription-id>'
region = 'eastus'

project = 'testopenai'
region = 'eastus'

environment = '-dev-'
resource_group_name = 'rg-' + project + environment + region + '-001' # create names using convention
openai_name = 'oai-' + project + environment + region + '-001'
deployment_name = 'gpt-' + project + environment + region + '-001'

In [3]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, subscription_id) # instanciate client object
cognitive_client = CognitiveServicesManagementClient(credential=AzureCliCredential(), subscription_id=subscription_id)

In [4]:
cognitive_account = cognitive_client.accounts.get(resource_group_name = resource_group_name,
                                                  account_name = openai_name)

### Call API

In [5]:
import openai

In [6]:
key = cognitive_client.accounts.list_keys(resource_group_name = resource_group_name,
                                                                 account_name = openai_name).key1
endpoint = cognitive_account.properties.endpoint

In [7]:
openai.api_type = "azure"
openai.api_base = endpoint
openai.api_version = "2022-12-01"
openai.api_key = key

In [8]:
class Encoder():
    def __init__(self, 
                 name: str, 
                 base: str = 'cl100k_base', 
                 start_token : str = '<|im_start|>',
                 end_token: str = '<|im_end|>'):
        self.start_token = start_token
        self.end_token = end_token
        self.cl100k_base = tiktoken.get_encoding("cl100k_base")
        self.encoder = tiktoken.Encoding(name=name,  
                                         pat_str=self.cl100k_base._pat_str, 
                                         mergeable_ranks=self.cl100k_base._mergeable_ranks, 
                                         special_tokens={ **self.cl100k_base._special_tokens, 
                                                         start_token: 100264, 
                                                         end_token: 100265})

In [9]:
class Prompt():
    def __init__(self, system_message: str, encoder: Encoder, max_tokens: int = 2040, verbose: bool = False):
        self.start_token = encoder.start_token
        self.end_token = '\n' + encoder.end_token + '\n'
        self.prompt = self.start_token + 'system\n' + system_message + self.end_token
        self.tokens_size = 0
        self.max_tokens = max_tokens
        self.tokenizer = encoder
        self.verbose = verbose
        self._update()
        
    def _update(self):
        self.tokens = self.tokenizer.encoder.encode(self.prompt, allowed_special={"<|im_start|>", "<|im_end|>"} )
        self.tokens_size = len(self.tokens)
        if self.tokens_size > self.max_tokens:
            if self.verbose:
                print(f'tokens size limit reached, actual size is {self.tokens_size} and limit is {self.max_tokens}')
                print('Removing oldests messages')
            self.remove_oldest_message()
               
        
    def add_message(self, message: str, speaker : str = 'user') -> None:
        if speaker == 'user':
            self.prompt = (self.prompt 
                           + self.start_token + speaker + '\n' 
                           + message 
                           + self.end_token 
                           + self.start_token + 'assistant\n')
        else:
            self.prompt = self.prompt + message + self.end_token
        self._update()
        
    def remove_oldest_message(self):
        start = self.prompt.find(self.start_token + 'user')
        middle = self.prompt.find('assistant\n')
        end = self.prompt.find(self.end_token, middle)
        to_be_removed = self.prompt[start : end + len(self.end_token)]
        if self.verbose: print(f'removing "{to_be_removed}"\n')
        self.prompt = self.prompt.replace(to_be_removed, '')
        self._update()
        
    def get_last_response(self):
        start = self.prompt.rfind(self.start_token + 'assistant')
        end = self.prompt.find(self.end_token, start)
        return self.prompt[start + len(self.start_token): end]
        
    def print_last_response(self):
        print(self.get_last_response())

In [10]:
class Chat():
    def __init__(self, engine: str, prompt: Prompt, temperature: int = 0, max_tokens: int = 2040, top_p: float = 0.5):
        self.engine = engine
        self.prompt = prompt
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.top_p = top_p
        self.stop = prompt.end_token.strip()
        
    def send_message(self):
        response = openai.Completion.create(
                                      engine=self.engine,
                                      prompt=self.prompt.prompt,
                                      temperature=self.temperature,
                                      max_tokens=self.max_tokens,
                                      top_p=self.top_p,
                                      stop=[self.stop])
        self.prompt.add_message(response['choices'][0]['text'], speaker='assistant')

### Geração de massa de dados

In [11]:
max_tokens = 2000

cl100k_encoder = Encoder(name = deployment_name)
system_message = 'O assitente ajuda a gerar dados seguindo um esquema definido.'
test_prompt = Prompt(system_message, encoder = cl100k_encoder, max_tokens = max_tokens)
test_chat = Chat(engine = deployment_name, prompt = test_prompt, max_tokens = max_tokens)

In [12]:
test_prompt.add_message("Temos uma tabela com 7 colunas: nome, idade, endereço, cidade, estado, CEP e CPF. \
                         A coluna nome é do tipo VARCHAR e tem o nome completo dos usuários. \
                         A coluna idade é do tipo INT e contém a idade dos usuários, \
                         A coluna endereço é do tipo VARCHAR e tem o endereço (rua, número e complemento) do usuário.\
                         A coluna cidade é to tipo VARCHAR e tem a cidade do usuário.\
                         A coluna estado é do tipo CHAR(2) e tem a sigla do estado do usuário. \
                         A coluna CEP é do tipo CHAR(9) e tem o CEP do usuário.\
                         A coluna CEP é do tipo CHAR(11) e tem o CPF do usuário.\
                         Gere 10 linhas de dados sintético seguindo esse esquema")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Segue abaixo 10 linhas de dados sintéticos seguindo o esquema definido:

| nome                 | idade | endereço                       | cidade           | estado | CEP         | CPF           |
|----------------------|-------|--------------------------------|------------------|--------|-------------|---------------|
| Ana Paula Santos     | 28    | Rua das Flores, 123            | São Paulo        | SP     | 01234-567  | 123.456.789-10 |
| João da Silva        | 42    | Rua dos Pinheiros, 456         | Rio de Janeiro   | RJ     | 23456-789  | 234.567.890-11 |
| Maria Aparecida Lima | 35    | Avenida Paulista, 789          | São Paulo        | SP     | 34567-890  | 345.678.901-12 |
| Pedro Henrique Sousa | 21    | Rua das Palmeiras, 987         | Belo Horizonte   | MG     | 45678-901  | 456.789.012-13 |
| Juliana Oliveira     | 29    | Rua dos Girassóis, 654         | Brasília         | DF     | 56789-012  | 567.890.123-14 |
| Rafaela Santos       | 27    | Rua das Acácias,

In [13]:
test_prompt.add_message("Escreva uma query SQL para inserir esses dados na tabela, o nome da tabela é cadastro_usuario")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Segue abaixo a query SQL para inserir os dados na tabela "cadastro_usuario":

```
INSERT INTO cadastro_usuario (nome, idade, endereco, cidade, estado, CEP, CPF)
VALUES 
('Ana Paula Santos', 28, 'Rua das Flores, 123', 'São Paulo', 'SP', '01234-567', '123.456.789-10'),
('João da Silva', 42, 'Rua dos Pinheiros, 456', 'Rio de Janeiro', 'RJ', '23456-789', '234.567.890-11'),
('Maria Aparecida Lima', 35, 'Avenida Paulista, 789', 'São Paulo', 'SP', '34567-890', '345.678.901-12'),
('Pedro Henrique Sousa', 21, 'Rua das Palmeiras, 987', 'Belo Horizonte', 'MG', '45678-901', '456.789.012-13'),
('Juliana Oliveira', 29, 'Rua dos Girassóis, 654', 'Brasília', 'DF', '56789-012', '567.890.123-14'),
('Rafaela Santos', 27, 'Rua das Acácias, 321', 'Salvador', 'BA', '67890-123', '678.901.234-15'),
('Lucas Pereira', 31, 'Avenida Atlântica, 987', 'Rio de Janeiro', 'RJ', '78901-234', '789.012.345-16'),
('Fernanda Souza', 24, 'Rua das Orquídeas, 456', 'São Paulo', 'SP', '89012-345', '890.123.456-17'),


### Transformar texto livre em dado estruturado

In [24]:
max_tokens = 2000

cl100k_encoder = Encoder(name = deployment_name)
system_message = 'O assistente recebe um campo de texto livre com informacões sobre veículos, \
                  nessas informações temos fabricante, modelo do veículo, versão do veículo,\
                  ano de inicio e ano de fim. Os dados devem ser inseridos em um tabela com as \
                  colunas fabricante, modelo, versão, ano.\
                  user: Focus - 1.6/ 1.8 / 2.0 - 01/08-08 \
                  assistente:\
                  | fabricante  | modelo     | versão    | ano | \
                  | Ford        | Focus      | 1.6       | 08  | \
                  | Ford        | Focus      | 1.8       | 08  | \
                  | Ford        | Focus      | 2.0       | 08  |'
test_prompt = Prompt(system_message, encoder = cl100k_encoder, max_tokens = max_tokens)
test_chat = Chat(engine = deployment_name, prompt = test_prompt, max_tokens = max_tokens)

In [25]:
test_prompt.add_message("CHEVROLET CORSA 2003 | TODOS OS MODELOS (NOVO CORSA) - CHEVROLET CORSA 2004 | \
                         TODOS OS MODELOS (NOVO CORSA) - CHEVROLET CORSA 2005 | TODOS OS MODELOS ")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Com base nas informações fornecidas, a tabela ficaria assim:

| fabricante  | modelo     | versão    | ano |
|-------------|------------|-----------|-----|
| Chevrolet   | Corsa      | Todos     | 2003|
| Chevrolet   | Corsa      | Todos     | 2004|
| Chevrolet   | Corsa      | Todos     | 2005|


In [26]:
test_prompt.add_message("Escreva uma query SQL para inserir os dados da ultima consulta \
                         em uma tabela chamada é aplicacao ")
test_chat.send_message()
test_prompt.print_last_response()

assistant
A query SQL para inserir os dados da última consulta na tabela "aplicacao" seria:

```
INSERT INTO aplicacao (fabricante, modelo, versao, ano)
VALUES 
('Chevrolet', 'Corsa', 'Todos', '2003'),
('Chevrolet', 'Corsa', 'Todos', '2004'),
('Chevrolet', 'Corsa', 'Todos', '2005');
```

Lembrando que é necessário ter a tabela "aplicacao" previamente criada com as colunas "fabricante", "modelo", "versao" e "ano".


In [27]:
test_prompt.add_message("NISSAN LIVINA 2010 | 1.8 (TRANSMISSÃO MANUAL) - NISSAN LIVINA 2011 | \
                         1.8 (TRANSMISSÃO MANUAL) - NISSAN LIVINA 2012 | 1.8 (TRANSMISSÃO MANUAL) - NISSAN ")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Com base nas informações fornecidas, a tabela ficaria assim:

| fabricante  | modelo     | versão           | ano |
|-------------|------------|-----------------|-----|
| Nissan      | Livina     | 1.8 (Manual)     | 2010|
| Nissan      | Livina     | 1.8 (Manual)     | 2011|
| Nissan      | Livina     | 1.8 (Manual)     | 2012|

Observação: Como não foi informado o modelo exato da Nissan Livina, utilizei apenas "Livina" na coluna "modelo".


In [28]:
test_prompt.add_message("Escreva uma query SQL para inserir esses dados na tabela, o nome da tabela é aplicacao ")
test_chat.send_message()
test_prompt.print_last_response()

assistant
A query SQL para inserir os dados da última consulta na tabela "aplicacao" seria:

```
INSERT INTO aplicacao (fabricante, modelo, versao, ano)
VALUES 
('Nissan', 'Livina', '1.8 (Manual)', '2010'),
('Nissan', 'Livina', '1.8 (Manual)', '2011'),
('Nissan', 'Livina', '1.8 (Manual)', '2012');
```

Lembrando que é necessário ter a tabela "aplicacao" previamente criada com as colunas "fabricante", "modelo", "versao" e "ano".



In [29]:
test_prompt.add_message("RENAULT Clio 01/98; Express 01/93 - 12/01;  Megane 01/97 - 12/05; R19 01/92 - 12/96 ")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Com base nas informações fornecidas, a tabela ficaria assim:

| fabricante  | modelo     | versão    | ano |
|-------------|------------|-----------|-----|
| Renault     | Clio       | Todos     | 1998|
| Renault     | Express    | Todos     | 1993|
| Renault     | Express    | Todos     | 2001|
| Renault     | Megane     | Todos     | 1997|
| Renault     | Megane     | Todos     | 2005|
| Renault     | R19        | Todos     | 1992|
| Renault     | R19        | Todos     | 1996|

Observação: Como não foi informado a versão exata dos modelos Renault Clio, Express, Megane e R19, utilizei "Todos" na coluna "versão".



In [30]:
test_prompt.add_message("Escreva uma query SQL para inserir esses dados na tabela, o nome da tabela é aplicacao ")
test_chat.send_message()
test_prompt.print_last_response()

assistant
A query SQL para inserir os dados da última consulta na tabela "aplicacao" seria:

```
INSERT INTO aplicacao (fabricante, modelo, versao, ano)
VALUES 
('Renault', 'Clio', 'Todos', '1998'),
('Renault', 'Express', 'Todos', '1993'),
('Renault', 'Express', 'Todos', '2001'),
('Renault', 'Megane', 'Todos', '1997'),
('Renault', 'Megane', 'Todos', '2005'),
('Renault', 'R19', 'Todos', '1992'),
('Renault', 'R19', 'Todos', '1996');
```

Lembrando que é necessário ter a tabela "aplicacao" previamente criada com as colunas "fabricante", "modelo", "versao" e "ano".


In [31]:
test_prompt.add_message("FORD Focus 1.6i 16V Flex 2010 ATÉ 2013 \
                         FORD Focus 2.0i 16V Flex 2010 ATÉ 2013 \
                         FORD Focus Sedan 1.6i 16V Flex 2010 ATÉ 2013 \
                         FORD Focus Sedan 2.0i 16V Flex 2010 ATÉ 2013 ")
test_chat.send_message()
test_prompt.print_last_response()

assistant
Com base nas informações fornecidas, a tabela ficaria assim:

| fabricante  | modelo        | versão              | ano |
|-------------|---------------|---------------------|-----|
| Ford        | Focus         | 1.6i 16V Flex       | 2010|
| Ford        | Focus         | 1.6i 16V Flex       | 2011|
| Ford        | Focus         | 1.6i 16V Flex       | 2012|
| Ford        | Focus         | 1.6i 16V Flex       | 2013|
| Ford        | Focus         | 2.0i 16V Flex       | 2010|
| Ford        | Focus         | 2.0i 16V Flex       | 2011|
| Ford        | Focus         | 2.0i 16V Flex       | 2012|
| Ford        | Focus         | 2.0i 16V Flex       | 2013|
| Ford        | Focus Sedan   | 1.6i 16V Flex       | 2010|
| Ford        | Focus Sedan   | 1.6i 16V Flex       | 2011|
| Ford        | Focus Sedan   | 1.6i 16V Flex       | 2012|
| Ford        | Focus Sedan   | 1.6i 16V Flex       | 2013|
| Ford        | Focus Sedan   | 2.0i 16V Flex       | 2010|
| Ford        | Focus Sedan 

In [32]:
test_prompt.add_message("Escreva uma query SQL para inserir esses dados na tabela, o nome da tabela é aplicacao ")
test_chat.send_message()
test_prompt.print_last_response()

assistant
A query SQL para inserir os dados da última consulta na tabela "aplicacao" seria:

```
INSERT INTO aplicacao (fabricante, modelo, versao, ano)
VALUES 
('Ford', 'Focus', '1.6i 16V Flex', '2010'),
('Ford', 'Focus', '1.6i 16V Flex', '2011'),
('Ford', 'Focus', '1.6i 16V Flex', '2012'),
('Ford', 'Focus', '1.6i 16V Flex', '2013'),
('Ford', 'Focus', '2.0i 16V Flex', '2010'),
('Ford', 'Focus', '2.0i 16V Flex', '2011'),
('Ford', 'Focus', '2.0i 16V Flex', '2012'),
('Ford', 'Focus', '2.0i 16V Flex', '2013'),
('Ford', 'Focus Sedan', '1.6i 16V Flex', '2010'),
('Ford', 'Focus Sedan', '1.6i 16V Flex', '2011'),
('Ford', 'Focus Sedan', '1.6i 16V Flex', '2012'),
('Ford', 'Focus Sedan', '1.6i 16V Flex', '2013'),
('Ford', 'Focus Sedan', '2.0i 16V Flex', '2010'),
('Ford', 'Focus Sedan', '2.0i 16V Flex', '2011'),
('Ford', 'Focus Sedan', '2.0i 16V Flex', '2012'),
('Ford', 'Focus Sedan', '2.0i 16V Flex', '2013');
```

Lembrando que é necessário ter a tabela "aplicacao" previamente criada com as colu